In [7]:
# from bs4 import BeautifulSoup
# import requests
import time
from bin.scrape import Listing_scraper, makes_dict_load
import pickle


In [5]:
makes_dict = makes_dict_load()  #supposed num of listings per make

new_makes_dict = {} #dict with no duplicate makes
for k, v in makes_dict.items():
    k = str(k).lower()
    if k in new_makes_dict.keys():
        continue
    else:
        new_makes_dict[k] = v

In [8]:
# makes_dict = makes_dict_load()  #supposed num of listings per make
pages_dict = {}                 #num of pages per make
links_dict = {}                 #links to collected listings per make 

##iterate over makes
for make, v in new_makes_dict.items():
    print(make)
    time.sleep(1)
    #get num of pages with listings for given make
    num = Listing_scraper(make).num_pages()

    ## if there seems to be only one page of listings, but there is supposed to be more than 32 listings for a make,
    ## reload the page 3 times, hoping it automagically works and shows multiple pages :)
    if num == 0 and int(v.replace(' ', '')) > 32:
        n = 0
        while n < 3:
            time.sleep(1)
            num = Listing_scraper(make).num_pages()
            
            if num != 0:
                break
            n += 1

    pages_dict[make] = num

    if num == 0: ##if there is only 1 page of listings for a make
        listings = Listing_scraper(make)
        a = listings.listing_links()
        links_dict[make] = a
    elif num > 0: ## if there is more than 1 page...
        a = [] #cummulative list of collected listing links for a make

        for i in range(num): #iterate over pages with listings of a make

            if i == 0: #define url for a given page
                url = make
            else:
                url=f'{make}?page={i+1}'

            time.sleep(1)
            
            try:
                listings = Listing_scraper(url)
                #checks if num of listings per make is still in the correct range
                if (0.5 * int(v.replace(' ', ''))) <= listings.check_num_of_listings() <= (1.5 * int(v.replace(' ', ''))):
                    b = listings.listing_links()
                    a = a + b

                else: #retry up to 3 times if the num of listings isn't correct
                    n = 0
                    while n < 3:
                        time.sleep(1)
                        listings = Listing_scraper(url)
                        num_check = listings.check_num_of_listings()
                        
                        if (0.9 * int(v.replace(' ', ''))) <= num_check <= (1.1 * int(v.replace(' ', ''))):
                            break
                        n += 1

                    b = listings.listing_links()
                    a = a + b
            except:
                continue

        links_dict[make] = a

    print('collected:' + str(len(a)) + '   expected: ' + str(v))

with open('links_2022-06-12.pickle', 'wb') as f:
    pickle.dump(links_dict, f, pickle.HIGHEST_PROTOCOL)

https://www.otomoto.pl/osobowe/acura/
collected:20   expected: 16
https://www.otomoto.pl/osobowe/aixam/
collected:157   expected: 151
https://www.otomoto.pl/osobowe/alpine/
collected:4   expected: 5
https://www.otomoto.pl/osobowe/audi/
collected:13837   expected: 15 397
https://www.otomoto.pl/osobowe/austin/
collected:9   expected: 10
https://www.otomoto.pl/osobowe/bmw/
collected:13555   expected: 15 288
https://www.otomoto.pl/osobowe/bentley/
collected:72   expected: 83
https://www.otomoto.pl/osobowe/buick/
collected:25   expected: 23
https://www.otomoto.pl/osobowe/cadillac/
collected:118   expected: 154
https://www.otomoto.pl/osobowe/chatenet/
collected:18   expected: 9
https://www.otomoto.pl/osobowe/chevrolet/
collected:1605   expected: 1 369
https://www.otomoto.pl/osobowe/chrysler/
collected:659   expected: 668
https://www.otomoto.pl/osobowe/cupra/
collected:116   expected: 108
https://www.otomoto.pl/osobowe/dfsk/
collected:18   expected: 14
https://www.otomoto.pl/osobowe/dkw/
coll